In [32]:
import pandas as pd
import re

In [33]:
file_1 = '1-1-2022n.xlsx'
file_2 = '2-1-2022n.xlsx'
file_3 = '3-1-2022n.xlsx'

In [34]:
FULLWIDTH_TO_HALFWIDTH = str.maketrans('１２３４５６７８９０',
                                       '1234567890')
def fullwidth_to_halfwidth(s):
    return s.translate(FULLWIDTH_TO_HALFWIDTH)

In [53]:
def gen_1_2(file_1, file_2, month, detCol, xlsxName=''):
    opMonth = pd.to_datetime(month+'.01', format='%Y.%m.%d').date()

    # load file_2 data
    df_2 = pd.read_excel(file_2, sheet_name=month, skiprows=5, header=None)
    df_2 = df_2[:df_2[df_2[2] == '合計'].index.values[0]]
    df_2.fillna(0, inplace=True)

    # load file_1 data
    df_1 = pd.read_excel(file_1, sheet_name=month, skiprows=5, header=None)
    df_1 = df_1[:df_1[df_1[2] == '合計'].index.values[0]]
    df_1 = df_1.replace(r'^\s*$', 0, regex=True)
    df_1.fillna(0, inplace=True)

    # process file 1 header
    df_1_header = pd.read_excel(file_1, sheet_name=month, header=None, nrows=4)
    str_dt = fullwidth_to_halfwidth(df_1_header.iloc[0, 49])

    m = re.match('(\d+)\D+(\d+)\D+(\d+)\D+', str_dt)
    publishDate_1 = pd.to_datetime(
        '-'.join(list(m.groups())), format='%Y-%m-%d').date()

    df_1_header.fillna(method='ffill', inplace=True)  # fill vertically for na
    df_1_header = df_1_header[1:]
    df_1_header.reset_index(drop=True, inplace=True)

    lst_dicts = []
    for ridx, row in df_1.iterrows():
        companyName = ''
        if row[detCol] != '○':
            continue
        companyName = row[2]
        j = 10  # for df_2 col index
        for i in range(10, 50, 2):
            d1 = {}
            d1['OpMonth'] = opMonth
            d1['CompanyNameLocal'] = companyName
            d1['FuelType'] = df_1_header.iloc[0, i]
            d1['GenerationType'] = df_1_header.iloc[1, i]
            d1['NumberOfPlant'] = row[i]
            d1['MaxOutputMW'] = row[i+1]/1000
            d1['GenMW'] = df_2.iloc[ridx, j]
            d1['PublishDate'] = publishDate_1
            lst_dicts.append(d1)
            j += 1
        # break

    df_result = pd.DataFrame(lst_dicts)
    if df_result.shape[0] > 0:
        df_result = df_result[df_result['FuelType'] != '計']
    if xlsxName:
        df_result.to_excel('output/{}.xlsx'.format(xlsxName), index=False)

    return df_result

In [56]:
def gen_3(file_3, month, xlsxName=''):
    opMonth = pd.to_datetime(month+'.01', format='%Y.%m.%d').date()

    df_3 = pd.read_excel(file_3, sheet_name=month, skiprows=5, header=None)
    df_3 = df_3[df_3[0].notna()]
    df_3.fillna(0, inplace=True)

    df_3_header = pd.read_excel(file_3, sheet_name=month, header=None, nrows=4)
    str_dt = fullwidth_to_halfwidth(df_3_header.iloc[0, 21])

    m = re.match('(\d+)\D+(\d+)\D+(\d+)\D+', str_dt)
    publishDate_3 = pd.to_datetime(
        '-'.join(list(m.groups())), format='%Y-%m-%d').date()

    df_3_header.fillna(method='ffill', inplace=True)  # fill vertically for na
    df_3_header = df_3_header[1:]
    df_3_header.reset_index(drop=True, inplace=True)

    lst_dicts = []
    for ridx, row in df_3.iterrows():
        companyName = ''
        # print(row)
        companyName = row[2]

        str_bits = ''
        for i in range(3, 9):
            if row[i] == '○':
                str_bits += '1'
            else:
                str_bits += '0'

        for i in range(10, 22):
            d1 = {}
            d1['OpMonth'] = opMonth
            d1['CompanyName'] = companyName
            d1['CompanyType'] = str_bits
            d1['FuelType'] = df_3_header.iloc[0, i]
            d1['GenerationType'] = df_3_header.iloc[1, i]
            # d1['DemandType'] = df_3_header.iloc[5, i]
            d1['DemandMW'] = row[i]
            d1['PublishDate'] = publishDate_3
            lst_dicts.append(d1)
    df_result = pd.DataFrame(lst_dicts)

    if xlsxName:
        df_result.to_excel('output/{}.xlsx'.format(xlsxName), index=False)

    return df_result


In [66]:
def gen_all_12(file_1, file_2, outfile=''):
    xl = pd.ExcelFile(file_1)
    lst_months = list(filter(lambda n: re.match(r'^20.*\d+$', n), xl.sheet_names))
    lst_df_12 = []
    for m in lst_months:
        for det in range(3, 10):
            print(m, det)
            lst_df_12.append(gen_1_2(file_1, file_2, m, det))

    df_12 = pd.concat(lst_df_12)
    if outfile:
        df_12.to_csv(outfile, index=False)
    return df_12

In [67]:
def gen_all_3(file_3, outfile=''):
  xl = pd.ExcelFile(file_3)
  lst_months = list(filter(lambda n: re.match(r'^20.*\d+$', n), xl.sheet_names))
  lst_df_3 = []
  for m in lst_months:
      print(m)
      lst_df_3.append(gen_3(file_3, m))

  df_3 = pd.concat(lst_df_3)
  if outfile:
    df_3.to_csv(outfile, index=False)
  return df_3

In [68]:
gen_all_12(file_1, file_2, '12.csv')

2022.4 3
2022.4 4
2022.4 5
2022.4 6
2022.4 7
2022.4 8
2022.4 9
2022.5 3
2022.5 4
2022.5 5
2022.5 6
2022.5 7
2022.5 8
2022.5 9
2022.6 3
2022.6 4
2022.6 5
2022.6 6
2022.6 7
2022.6 8
2022.6 9
2022.7 3
2022.7 4
2022.7 5
2022.7 6
2022.7 7
2022.7 8
2022.7 9
2022.8 3
2022.8 4
2022.8 5
2022.8 6
2022.8 7
2022.8 8
2022.8 9
2022.9 3
2022.9 4
2022.9 5
2022.9 6
2022.9 7
2022.9 8
2022.9 9
2022.10 3
2022.10 4
2022.10 5
2022.10 6
2022.10 7
2022.10 8
2022.10 9


,OpMonth,CompanyNameLocal,FuelType,GenerationType,NumberOfPlant,MaxOutputMW,GenMW,PublishDate
0,2022-04-01,北海道電力株式会社,水力発電所,一般,51.0,832.57,323945,2022-08-09
1,2022-04-01,北海道電力株式会社,水力発電所,揚水式,3.0,800.00,64064.0,2022-08-09
2,2022-04-01,北海道電力株式会社,水力発電所,小計,54.0,1632.57,388010.0,2022-08-09
3,2022-04-01,北海道電力株式会社,火力発電,石炭,3.0,2250.00,606017,2022-08-09
4,2022-04-01,北海道電力株式会社,火力発電,ＬＮＧ,1.0,569.40,308620.0,2022-08-09
...,...,...,...,...,...,...,...,...
514,2022-10-01,Ｇｏａｌ ｃｏｎｎｅｃｔ株式会社,新エネルギー等発電所,地熱,0.0,0.00,0.0,2023-02-02
515,2022-10-01,Ｇｏａｌ ｃｏｎｎｅｃｔ株式会社,新エネルギー等発電所,（再掲）バイオマス,0.0,0.00,0.0,2023-02-02
516,2022-10-01,Ｇｏａｌ ｃｏｎｎｅｃｔ株式会社,新エネルギー等発電所,（再掲）廃棄物,0.0,0.00,0.0,2023-02-02
517,2022-10-01,Ｇｏａｌ ｃｏｎｎｅｃｔ株式会社,新エネルギー等発電所,小計,0.0,0.00,0.0,2023-02-02


In [69]:
gen_all_3(file_3, '3.csv')

2022.4
2022.5
2022.6
2022.7
2022.8
2022.9
2022.10


,OpMonth,CompanyName,CompanyType,FuelType,GenerationType,DemandMW,PublishDate
0,2022-04-01,北海道電力株式会社,100001,その他需要(自由料金),計,1.476398e+06,2022-08-09
1,2022-04-01,北海道電力株式会社,100001,その他需要(自由料金),特別高圧,2.458440e+05,2022-08-09
2,2022-04-01,北海道電力株式会社,100001,その他需要(自由料金),高圧,7.383200e+05,2022-08-09
3,2022-04-01,北海道電力株式会社,100001,その他需要(自由料金),低圧,4.922340e+05,2022-08-09
4,2022-04-01,北海道電力株式会社,100001,その他需要(自由料金),(再掲)低圧電灯,3.686560e+05,2022-08-09
...,...,...,...,...,...,...,...
8599,2022-10-01,(再掲)みなし小売電気事業者以外計,000000,特定需要(経過措置料金),(再掲)低圧電灯,0.000000e+00,2023-02-02
8600,2022-10-01,(再掲)みなし小売電気事業者以外計,000000,特定需要(経過措置料金),(再掲)低圧電力,0.000000e+00,2023-02-02
8601,2022-10-01,(再掲)みなし小売電気事業者以外計,000000,最終保障供給,最終保障供給,0.000000e+00,2023-02-02
8602,2022-10-01,(再掲)みなし小売電気事業者以外計,000000,離島供給,離島供給,0.000000e+00,2023-02-02
